In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2

# Fisher's method combines p-values, so we need to convert significances (z-scores) to p-values
def z_to_p(z):
    return 2 * (1 - norm.cdf(abs(z)))

# Fisher's method to combine p-values
def fishers_method(p_values):
    chi_stat = -2 * np.sum(np.log(p_values))
    dof = 2 * len(p_values)
    combined_p = chi2.sf(chi_stat, dof)
    return chi_stat, combined_p

# Original and corrected z-scores
z_corrected = [-3.3187620143735597, 0.26001531179730614, 2.075586664428067, -4.836001257130424, -5.0058209698922065, -4.124807725419037, -6.708145430556573, -0.36453333603403965, 1.0099999999999996, 0.4100000000000001, 1.0800000000000003, 0.8899999999999998, 2.039999999999999, 0.0, 0.21000000000000002, 0.0, 0.8000000000000002, 0.0, 0.9700000000000002, 0.0, 0.0, 0.0, 2.7200000000000046, 1.6699999999999997, 1.0599999999999998, 0.5100000000000001, 0.5400000000000001, 0.0, 0.0, 2.549999999999998, 1.5600000000000003, 2.8700000000000054, 0.0, 0.0, 0.0, 0.0, 0.6100000000000002, 0.53, 0.48, 0.0, 0.0, 0.0, 0.24999999999999997, -2.5933070558261178, -0.8292210803671793, 1.4830717993535725, -5.302971520631429, -5.19865894117264, -0.6987210437981668, 1.776667592584381, -2.9760254098401253, -1.7192364133998934, -1.5341205443525463, 1.8000000000000003, 2.0100000000000007, 0.5400000000000001, 1.7899999999999998, 1.0200000000000002, 1.7300000000000006, 0.2999999999999998, 0.0, 0.26000000000000006, 0.19000000000000014, 0.009999999999999985, 1.0099999999999996, 0.0, 0.8300000000000002, 0.059999999999999984, 1.2200000000000002, 0.21999999999999992, 2.8999999999999986, 0.7400000000000003, 0.7200000000000001, 0.0, 0.27999999999999986, 0.0, 0.0, 2.200000000000001, 0.0, 0.0, 1.209999999999999, 0.0, 0.66, 1.2599999999999998, 2.1100000000000003, 1.5100000000000002, 1.7700000000000005, 0.0, 0.0, 0.98]
z_original = [0.0, 1.7, 2.22, 0.27, 0.0, 0.57, 0.0, 0.75, 1.01, 0.41, 1.08, 0.89, 2.04, 0.0, 0.21, 0.0, 0.8, 0.0, 0.97, 0.0, 0.0, 0.0, 2.72, 1.67, 1.06, 0.51, 0.54, 0.0, 0.0, 2.55, 1.56, 2.87, 0.0, 0.0, 0.0, 0.0, 0.61, 0.53, 0.48, 0.0, 0.0, 0.0, 0.25, 0.3, 1.11, 1.66, 0.24, 0.17, 1.53, 3.04, 0.98, 0.92, 0.0, 1.8, 2.01, 0.54, 1.79, 1.02, 1.73, 0.3, 0.0, 0.26, 0.19, 0.01, 1.01, 0.0, 0.83, 0.06, 1.22, 0.22, 2.9, 0.74, 0.72, 0.0, 0.28, 0.0, 0.0, 2.2, 0.0, 0.0, 1.21, 0.0, 0.66, 1.26, 2.11, 1.51, 1.77, 0.0, 0.0, 0.98]

# i=z_original.index(3.45)
# z_original.remove(z_original[i])
# z_corrected.remove(z_corrected[i])
# Convert to p-values
p_corrected = [z_to_p(z) for z in z_corrected]
p_original = [z_to_p(z) for z in z_original]

# Apply Fisher's method
chi_corr, p_comb_corr = fishers_method(p_corrected)
chi_orig, p_comb_orig = fishers_method(p_original)

# Create table
results_df = pd.DataFrame({
    "Set": ["Corrected", "Original"],
    "Chi^2": [chi_corr, chi_orig],
    "p-value": [p_comb_corr, p_comb_orig]
})


print('corrected',chi_corr, p_comb_corr,norm.isf(p_comb_corr))
print('original',chi_orig, p_comb_orig ,norm.isf(p_comb_orig))




corrected 399.5773058293334 1.0448014511284677e-18 8.752347230349724
original 191.4417006230236 0.26576551061413567 0.6256705994403441


In [6]:
1-z_to_p(1)

0.6826894921370859

In [ ]:
def significance_to_pvalue(z):
    return norm.sf(z)

def process_file(filepath, ref_psf):
    df = pd.read_csv()
    results = []
    for _, row in df.iterrows():
        name = row['Name']
        transit = row['transit']
        sig = row['Significance']
        psf = row['PSF']
        p_val = significance_to_pvalue(sig)
        pdf_val = norm.pdf(sig)
        trials_factor = (psf**2) / (ref_psf**2)
        p_corr = 1- np.power(p_val,trials_factor)
        sig_corr = norm.isf(p_corr) 
        results.append((name, transit, sig, sig_corr, p_val, 1-p_corr, pdf_val))
    return results

In [ ]:
z_original.index(3.45)


77

In [7]:
# libraries/ul_plot.py
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from libraries import dataframe_generator
from scipy.stats import norm
from astropy.io import fits

def non_to_int2(E1, E2, flux, alpha, pivot):
    """Convierte un flujo a un valor integrado según el índice espectral."""
    A = flux / (pivot ** (-alpha))
    if alpha > 2:
        F1 = (E2 ** (2 - alpha)) / (2 - alpha)
        F2 = (E1 ** (2 - alpha)) / (2 - alpha)
        return 1.6 * A * (F1 - F2)
    elif alpha == 2:
        F3 = np.log(E2)
        F4 = np.log(E1)
        return A * (F3 - F4)
    else:
        return None

def expected_value(values, weights):
    """Calcula el valor esperado dada una lista de valores y sus pesos."""
    values = np.asarray(values)
    weights = np.asarray(weights)
    return (values * weights).sum() / weights.sum()

def generate_upper_limits_plots(config):
    """
    Genera DataFrames, procesa los datos y produce gráficos con los límites superiores
    obtenidos por los métodos Franceschini y Gilmore.
    """
    # Usar 'BASE_DIR' si existe, de lo contrario usar 'PATH_BASE'
    BASE_DIR = config.get('BASE_DIR', config.get('PATH_BASE'))
    bin_size = config['bin_size']
    spectral_index = config['spectral_index']
    GRB_KN_DIR = config['GRB_KN_DIR']
    GITLAB_DIR = config['GITLAB_DIR']
    
    # Construir rutas a partir de directorios base
    csvfile = config['GRBsINFO']
    path_zenith = os.path.join(GITLAB_DIR, 'UpperLimits/CSV_FILES_trials/GRB_COORDINATES/zenith.txt')
    upper_limit_dir = os.path.join(GRB_KN_DIR, f'data/ULs/files/PSF_{bin_size}', f'alfa={spectral_index}')
    
    Franceschini_1stT = os.path.join(upper_limit_dir, 'UpperLimit_1_Franceschini08.csv')
    Franceschini_2ndT = os.path.join(upper_limit_dir, 'UpperLimit_2_Franceschini08.csv')
    Gilmore_1stT = os.path.join(upper_limit_dir, 'UpperLimit_1_Gilmore12Fiducial.csv')
    Gilmore_2ndT = os.path.join(upper_limit_dir, 'UpperLimit_2_Gilmore12Fiducial.csv')
    print(Franceschini_1stT, 'eta')
    energy_file = config['energy_ranges']
    
    plots_dir = config.get('plots_dir', '../plots')
    plot_first_path = os.path.join(plots_dir, 'plot_first_transit_hd.png')
    plot_second_path = os.path.join(plots_dir, 'plot_second_transit_hd.png')
    
    # Generar DataFrames usando el módulo dataframe_generator
    df_franceschini = dataframe_generator.DATAFRAME_generator(Franceschini_2ndT, Franceschini_1stT, csvfile, path_zenith)
    df_gilmore = dataframe_generator.DATAFRAME_generator(Gilmore_2ndT, Gilmore_1stT, csvfile, path_zenith)
    df_franceschini.to_csv('data.csv')
    df_gilmore.to_csv('data2.csv')
    # Limpieza de nombres, ordenación y eliminación de duplicados
    df_franceschini['Name'] = df_franceschini["Name"].str.replace(',', '', regex=True)
    df_gilmore['Name'] = df_gilmore["Name"].str.replace(',', '', regex=True)
    df_franceschini=df_franceschini.drop_duplicates()
    df_franceschini.sort_values(by='Name', inplace=True)
    df_gilmore.sort_values(by='Name', inplace=True)
    df_gilmore=df_gilmore.drop_duplicates()
    
    Names = df_franceschini["Name"]
    DEC = df_franceschini['Dec'].copy()
    # if len(DEC) > 13:
    #     DEC.iloc[13] = -23  # Ajuste manual si es necesario
    
    colors = plt.cm.tab20(np.linspace(0, 1, 20))[:14]
    color_hex = [matplotlib.colors.rgb2hex(color) for color in colors]
    
# This section of the code is performing the following tasks:
    # Cargar archivo de energía y convertir a numérico
    Energy = pd.read_csv(energy_file, header=None, names=['Name', 'E1', 'E2'])
    Energy.sort_values(by='Name', inplace=True)
    Energy[['E1', 'E2']] = Energy[['E1', 'E2']].apply(pd.to_numeric, errors='coerce')

    # Asegurar que los límites superiores también son numéricos
    df_franceschini['upperBound'] = pd.to_numeric(df_franceschini['upperBound'], errors='coerce')
    df_franceschini['upperBound_2ndT'] = pd.to_numeric(df_franceschini['upperBound_2ndT'], errors='coerce')
    df_gilmore['upperBound'] = pd.to_numeric(df_gilmore['upperBound'], errors='coerce')
    df_gilmore['upperBound_2ndT'] = pd.to_numeric(df_gilmore['upperBound_2ndT'], errors='coerce')

    # Calcular ULs integradas
    UL_Franceschini_1 = non_to_int2(Energy['E1'], Energy['E2'], df_franceschini['upperBound'], spectral_index, 1)
    UL_Franceschini_2 = non_to_int2(Energy['E1'], Energy['E2'], df_franceschini['upperBound_2ndT'], spectral_index, 1)
    UL_Gilmore_1 = non_to_int2(Energy['E1'], Energy['E2'], df_gilmore['upperBound'], spectral_index, 1)
    UL_Gilmore_2 = non_to_int2(Energy['E1'], Energy['E2'], df_gilmore['upperBound_2ndT'], spectral_index, 1)

    # Guardar por si acaso
    DATA = pd.DataFrame({'F': UL_Franceschini_1, 'G': UL_Gilmore_1})
    DATA.to_csv('UL.csv')

    def plot_ul(names, dec, ul_franceschini, ul_gilmore, color_hex,
                xlabel=r'Declinación [grados]',
                ylabel=r'$\Phi$ [ergs cm$^{-2}$ s$^{-1}$]',
                xlim=(-25, 65), ylim=(1e-13, 1e-8),
                save_path=None):
        fig, ax = plt.subplots()
        for grb, ul_g, d, col in zip(names, ul_gilmore, dec, color_hex):
            ax.errorbar(x=d, y=ul_g, yerr=ul_g / 10, uplims=True, color=col, label=grb)
        ax.set_yscale('log')
        for grb, ul_f, ul_g, d, col in zip(names, ul_franceschini, ul_gilmore, dec, color_hex):
            ax.fill_between([d - 1, d + 1], [ul_f, ul_f], [ul_g, ul_g], color=col, alpha=0.3)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        # ax.legend(bbox_to_anchor=(1.05, 1))
        if save_path is not None:
            plt.savefig(save_path, dpi=600, bbox_inches='tight')
            plt.close(fig)
        else:
            plt.show()
    
    plot_ul(Names, DEC, UL_Franceschini_1, UL_Gilmore_1, color_hex, save_path=plot_first_path)
    plot_ul(Names, DEC, UL_Franceschini_2, UL_Gilmore_2, color_hex, save_path=plot_second_path)


ModuleNotFoundError: No module named 'libraries'